__Compare TE FDFD solution with analytical solution__

Daniel Köhn 
Kiel, 01/09/2017

__Import Libraries__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
from copy import copy
import matplotlib.ticker as mtick
import scipy.special as sp
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pickle

__FD grid dimensions__

In [ ]:
DH = 0.07;
NX = 180;
NY = 180;

__Define fonts__

In [ ]:
FSize = 15
font = {'color':  'black',
        'weight': 'normal',
        'size': FSize,
        'family': 'serif'} 
rcParams['figure.figsize'] = 10, 10

__Read true P-wave velocity model__

In [ ]:
f1 = open ('../snap/2_cross_TE_shot_1_50MHz.p')
data_type = np.dtype ('float32').newbyteorder ('<')
p = np.fromfile (f1, dtype=data_type)
p = p.reshape(NX,NY)
p = np.transpose(p)
p = np.flipud(p)
p = p / np.amax(p)

__Define Axis__

In [ ]:
x = np.arange(DH, DH*(NX+1), DH)
y = np.arange(DH, DH*(NY+1), DH)
lim = min(x),max(x),min(y),max(y);

__Define Acquisition geometry__

Define source positions

In [ ]:
XSRC = 2.0
YSRC = 2.0

Calculate source-receiver distance r 

In [ ]:
xv, yv = np.meshgrid(x, y)
r = np.sqrt((xv-XSRC)**2+(yv-YSRC)**2)

Define discrete frequencies

In [ ]:
f = 50e6
w = 2.0 * np.pi * f

**Calculate analytical solution for homogeneous acoustic full space at each receiver position**

In [ ]:
# define material parameters 
mu0 = 4 * np.pi * 1e-7             # magnetic permeability
eps = 4.0 * 8.85e-12               # permittivity
sigma = 3e-3                       # conductivity
epse = eps + (((0+1j)*sigma) / w)  # effective permittivity

k = w * np.sqrt(mu0 * epse)        # complex wavenumber

# analytical solution
p_analy = -np.flipud(np.real(w * mu0 * sp.hankel1(0,(r*k)) / 4.0))
p_analy = p_analy / np.amax(p_analy)
vmin = -1.0
vmax = 1.0

__Plot SubPlots__

In [ ]:
ax1 = plt.subplot2grid((2,2), (0,0), colspan=2)

plt.plot(x,p_analy[:][150],'r-',label="analytical solution")
plt.plot(x,p[:][150],'b-',label="GERMAINE")

plt.legend(loc=4,prop={'weight':'normal','size':FSize,'family':'serif'})
plt.grid(True)
plt.autoscale(enable=True, axis='x', tight=True)

# apply font properties to tick labels
a = gca()
a.set_xticklabels(a.get_xticks(), font)
a.set_yticklabels(a.get_yticks(), font)

plt.title('GERMAINE vs. analytical solution (50 MHz)', fontdict=font)
plt.ylabel(r'$Re(E_y)$', fontdict=font)
plt.xlabel('Distance [m]', fontdict=font)

ax2 = plt.subplot2grid((2,2), (1, 0))
    
# imshow of Vp models
cs = plt.imshow(p_analy, cmap='seismic', interpolation='none', extent=[min(x),max(x),min(y),max(y)], vmin=vmin, vmax=vmax, aspect='auto')
    
# plot source and receiver positions
plt.plot(XSRC,YSRC,'r*',linewidth=3,markersize=15);
        
# display values below lower clip value vmin as white 
cs.cmap.set_under('w')
    
# limit imshow 
plt.axis(lim);
    
# apply font properties to tick labels
a = gca()
a.set_xticklabels(a.get_xticks(), font)
a.set_yticklabels(a.get_yticks(), font)
    
a.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0d'))
a.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0d'))
        
# label title and axis and apply font properties
plt.title('Analytical solution', fontdict=font)
plt.ylabel('Depth [m]', fontdict=font)
plt.xlabel('Distance [m]', fontdict=font)
plt.gca().invert_yaxis()

ax3 = plt.subplot2grid((2,2), (1, 1))
    
# imshow of Vp models
cs = plt.imshow(p, cmap='seismic', interpolation='none', extent=[min(x),max(x),min(y),max(y)], vmin=vmin, vmax=vmax, aspect='auto')
    
# plot source and receiver positions
plt.plot(XSRC,YSRC,'r*',linewidth=3,markersize=15);
        
# display values below lower clip value vmin as white 
cs.cmap.set_under('w')
    
# limit imshow 
plt.axis(lim);

ax3.set_yticks([ ])
    
# apply font properties to tick labels
a = gca()
a.set_xticklabels(a.get_xticks(), font)
a.set_yticklabels(a.get_yticks(), font)
    
a.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0d'))
a.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0d'))
        
# label title and axis and apply font properties
plt.title('GERMAINE', fontdict=font)
plt.xlabel('Distance [m]', fontdict=font)
plt.gca().invert_yaxis()

divider = make_axes_locatable(ax3)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(cs, cax=cax)
cbar.set_label(r"$Re(E_y)$", fontdict=font, labelpad=1)
cbar.ax.tick_params(labelsize=FSize) 

# optimize layout
plt.tight_layout()
#plt.savefig('test.png', format='png', dpi=100)
plt.savefig('GERMAINE_analy_comp.pdf', format='pdf')
plt.show()